# 1. Initialize robot and environment

## Load curobo

In [ ]:
# torch
import torch
import numpy as np

# cuRobo
from curobo.cuda_robot_model.cuda_robot_model import CudaRobotModel, CudaRobotModelConfig
from curobo.types.base import TensorDeviceType
from curobo.types.robot import RobotConfig
from curobo.util_file import get_robot_path, join_path, load_yaml
from robofab.mqtt import publish_robot_trajs, publish_clear, publish_world, publish_frames, publish_grasp_object
# convenience function to store tensor type and device
tensor_args = TensorDeviceType()

## Load franka

In [ ]:
# the path of the yml file must be absolute
from robofab import ROBOFAB_DATA_DIR
def load_franka_kin_model(gripper_width = 0.04):
    robot_name = "left"
    config_file = load_yaml(ROBOFAB_DATA_DIR+ f"/robot/dual_franka/fr3_franka_{robot_name}.yml")["robot_cfg"]
    config_kinematics = config_file['kinematics']
    config_kinematics["urdf_path"] = ROBOFAB_DATA_DIR + "/robot/dual_franka/" + config_kinematics["urdf_path"]
    config_kinematics["collision_spheres"] = ROBOFAB_DATA_DIR + "/robot/dual_franka/" + config_kinematics["collision_spheres"]
    config_kinematics["lock_joints"] = {"fr3_finger_joint1": gripper_width, "fr3_finger_joint2": gripper_width}

    robot_cfg = RobotConfig.from_dict(config_file, tensor_args)
    kin_model = CudaRobotModel(robot_cfg.kinematics)
    return config_file, kin_model

In [ ]:
_, left_kin_model = load_franka_kin_model(0.04)
retract_q = left_kin_model.retract_config.cpu().numpy().reshape(1, -1)
print(retract_q)
publish_clear()
publish_robot_trajs(retract_q, 0.04)

## Load environment

### load place environment

In [ ]:
from robofab import ROBOFAB_DATA_DIR
from robofab.mqtt import publish_world, publish_clear
from curobo.geom.types import WorldConfig, Mesh
import trimesh
def load_place_environment(part_ids = np.arange(12)):
    ground_obj_file = ROBOFAB_DATA_DIR + "/world/ground_with_platform.obj"
    assembly_objs_file = [ROBOFAB_DATA_DIR + f"/world/tetris/{id}.obj" for id in part_ids]

    ground_obstacle = Mesh(
        name=f"ground",
        pose=[0, 0, 0, 1, 0, 0, 0],
        file_path = ground_obj_file,
        scale = (1E-3, 1E-3, 1E-3)
    )

    assembly_obstacles = []
    assembly_meshes = []
    id = 0
    translation = [0.6, 0.1, -0.006]
    for file in assembly_objs_file:
        assembly_obstacles.append(Mesh(
            name=f"part_{id}",
            pose=[*translation, 1, 0, 0, 0],
            file_path = file,
            scale = (1, 1, 1)
        ))
        mesh = trimesh.load(file)
        mesh.apply_translation(translation)
        assembly_meshes.append(mesh)
        id = id + 1
            
    return assembly_meshes, WorldConfig(
       mesh=[*assembly_obstacles, ground_obstacle]
    )

assembly_meshes, place_world = load_place_environment([2])
_, ground_world = load_place_environment([])
publish_clear()
publish_robot_trajs(retract_q, 0.04)
publish_world(place_world, merged = True)

### load pick environment

In [ ]:
from robofab.pick import create_robot_pick_station
# shapes can be ["S-", "S+", "S2-", "T-", "T+", "T+2", "O", "L4-", "L4+", "L4|-", "L4|+", "I4"]
pick_world = create_robot_pick_station(voxel_size=0.05, shapes = ["T+"])[0]
publish_clear()
publish_robot_trajs(retract_q, 0.04)
publish_world(pick_world[3], merged = True)

# 2. Compute Place IK

## 2.1 Compute place poses

### Without predefining gripper z-axis

In [ ]:
from robofab.place import compute_part_place_pose
from robofab.place import poses_to_frames
place_poses = compute_part_place_pose(assembly_meshes[0], tool_offset = 0)

# render
frames = poses_to_frames(place_poses)
publish_frames(frames)

### Predefining the gripper z-axis

In [ ]:
place_poses_with_zaxis = compute_part_place_pose(assembly_meshes[0], tool_offset = 0, zaxis = np.array([0, 0, -1]))

# render
frames = poses_to_frames(place_poses)
publish_frames(frames)

## 2.2 Initialize IK Solver

### initialize IK solver function

In [ ]:
from curobo.geom.types import Pose
from curobo.wrap.reacher.ik_solver import IKSolver, IKSolverConfig

def get_franka_ik_solver(gripper_width=0.04, world_config: WorldConfig = None, num_seeds=50):
    config_file, _ = load_franka_kin_model(gripper_width)
    robot_cfg = RobotConfig.from_dict(config_file)
    ik_config = IKSolverConfig.load_from_robot_config(
            robot_cfg,
            world_model=world_config,
            tensor_args=tensor_args,
            use_cuda_graph=True,
            rotation_threshold=0.001,
            position_threshold=0.001,
            num_seeds=num_seeds,
            self_collision_check=True,
            self_collision_opt=True,
            collision_activation_distance= 0.02
        )
    ik_solver = IKSolver(ik_config)
    return ik_solver

### initialize Plan IK function

In [ ]:
def get_poses_by_flag(poses: Pose, flag: torch.tensor):
    return Pose(position=poses.position[flag, :], quaternion=poses.quaternion[flag, :])

def plan_ik(poses: Pose,
            world_config: WorldConfig,
            gripper_width: float = 0.04):
    ik_solver = get_franka_ik_solver(gripper_width=gripper_width, world_config=world_config)
    result = ik_solver.solve_batch(poses)
    ik_solver.reset_cuda_graph()
    if result.success.any():
        flag = result.success
        success_qs = result.solution[result.success]
        sucess_pose = get_poses_by_flag(poses, flag.flatten())
        return (success_qs, sucess_pose, flag.flatten())
    else:
        return None

### Initialize Plan IK with seed function

In [ ]:
from curobo.types.state import JointState
def plan_ik_seed_config(poses: Pose,
                        world_config: WorldConfig,
                        seed_config: JointState,
                        gripper_width: float = 0.04):
    ik_solver = get_franka_ik_solver(gripper_width=gripper_width, world_config=world_config, num_seeds = 1)
    seed_config = seed_config.position.view(1, -1)
    result = ik_solver.solve_batch(goal_pose=poses, seed_config=seed_config, num_seeds=1)
    if result.success.any():
        flag = result.success
        success_qs = result.solution[result.success]
        sucess_pose = get_poses_by_flag(poses, flag.flatten())
        return (success_qs, sucess_pose, flag.flatten())
    else:
        return None

# 2.3 Plan Place IK

### Plan place ik

In [ ]:
place_result = plan_ik(place_poses, place_world, 0.04)

# update
place_poses = place_result[1]

# render
publish_clear()
publish_robot_trajs(place_result[0].cpu().numpy(), 0.04)
frames = poses_to_frames(place_poses)
publish_frames(frames)
publish_world(place_world, merged = True)

### Plan place approach IK

In [ ]:
place_approach_poses = place_poses.clone()
place_approach_poses.position += torch.tensor([0, 0, 0.1], **tensor_args.as_torch_dict())
place_approach_result =  plan_ik(place_approach_poses, place_world, 0.04)

# update
place_approach_poses = place_approach_result[1]
place_poses = get_poses_by_flag(place_poses, place_approach_result[2])
print("Number of place poses", place_poses.position.shape[0])

# render
publish_clear()
publish_robot_trajs(place_approach_result[0].cpu().numpy(), 0.04)
frames = poses_to_frames(place_approach_poses)
publish_frames(frames)
publish_world(place_world, merged = True)

# 3. Compute Pick IK

# 3.1 Compute pick poses

### Compute place to pick transformation matrix

In [ ]:
from robofab.pick import compute_part_transformation_from_place_to_pick
# It can have multiple mats for achieving the same effect
place_to_pick_mats = compute_part_transformation_from_place_to_pick(assembly_meshes[0], pick_world, voxel_size = 0.05)

### Compute pick frames

In [ ]:
# render
publish_clear()
place_frames = poses_to_frames(place_poses)
pick_frames = place_to_pick_mats[0] @ place_frames
publish_frames(pick_frames)
publish_world(pick_world[3], merged = False)

### Compute pick poses

In [ ]:
from curobo.geom.transform import matrix_to_quaternion
pick_frames = torch.tensor(pick_frames, **tensor_args.as_torch_dict())
pick_poses = Pose(position=pick_frames[:, :3, 3], quaternion=matrix_to_quaternion(pick_frames[:, :3, :3]))

## 3.2 Compute pick IK

### Plan pick ik

In [ ]:
pick_result = plan_ik(pick_poses, place_world, 0.04)

# update
pick_poses = pick_result[1]
place_poses = get_poses_by_flag(place_poses, pick_result[2])
place_approach_poses = get_poses_by_flag(place_approach_poses, pick_result[2])

# render
publish_clear()
publish_robot_trajs(pick_result[0].cpu().numpy(), 0.04)
publish_world(pick_world[3], merged = True)

### Plan pick approach ik

In [ ]:
pick_approach_poses = pick_poses.clone()
pick_approach_poses.position += torch.tensor([0, 0, 0.1], **tensor_args.as_torch_dict())
pick_approach_result =  plan_ik(pick_approach_poses, pick_world[3], 0.04)

# update
pick_approach_poses = pick_approach_result[1]
pick_poses = get_poses_by_flag(pick_poses, pick_approach_result[2])
place_poses = get_poses_by_flag(place_poses, pick_approach_result[2])
place_approach_poses = get_poses_by_flag(place_approach_poses, pick_approach_result[2])
print("Number of pick poses", place_poses.position.shape[0])

# render
publish_clear()
publish_robot_trajs(pick_approach_result[0].cpu().numpy(), 0.04)
publish_world(pick_world[3], merged = True)

# 4. Plan Motions

## 4.1 Initialize motion solver

### Motion gen configuration

In [ ]:
from curobo.wrap.reacher.motion_gen import MotionGen, MotionGenConfig, MotionGenPlanConfig
from curobo.util.trajectory import InterpolateType

def get_franka_motion_gen(gripper_width=0.04, world_config: WorldConfig = None, num_seeds=50):
    config_file, _ = load_franka_kin_model(gripper_width)
    robot_cfg = RobotConfig.from_dict(config_file)
    motion_gen_config = MotionGenConfig.load_from_robot_config(
            robot_cfg,
            world_model=world_config,
            tensor_args=tensor_args,
            use_cuda_graph=True,
            rotation_threshold=0.05,
            position_threshold=0.005,
            num_ik_seeds=num_seeds,
            num_trajopt_seeds=num_seeds,
            interpolation_dt=0.01,
            interpolation_steps = 1000,
            interpolation_type=InterpolateType.CUBIC,
            high_precision=True,
            self_collision_check=True,
            self_collision_opt=True,
            minimize_jerk=True,
            collision_activation_distance=0.02)
    return MotionGen(motion_gen_config)

### Motion gen planner

In [ ]:
from curobo.types.state import JointState
def plan_motion_attached_object(start_state: JointState,
                                goal_poses: Pose,
                                attached_object: Mesh,
                                attached_state : JointState,
                                world_config: WorldConfig,
                                gripper_width: float = 0.04):
    
    motion_gen = get_franka_motion_gen(world_config=world_config, gripper_width=gripper_width)
    place_poses = Pose(position=goal_poses.position.view(1, -1, 3), quaternion=goal_poses.quaternion.view(1, -1, 4))
    start_state.joint_names = motion_gen.kinematics.joint_names.copy()

    attached_state.joint_names = motion_gen.kinematics.joint_names.copy()
    motion_gen.attach_external_objects_to_robot(attached_state,
                                                [attached_object],
                                                surface_sphere_radius=0.015,
                                                link_name="attached_object")

    result = motion_gen.plan_goalset(start_state, place_poses, MotionGenPlanConfig(max_attempts=10))
    if result.success.any():
        qtraj = result.get_interpolated_plan().position
        return (qtraj, result.goalset_index.item())
    else:
        return None

In [ ]:
def plan_motion_js_attached_object(start_state: JointState,
                                goal_state: JointState,
                                attached_object: Mesh,
                                attached_state : JointState,
                                world_config: WorldConfig,
                                gripper_width: float = 0.04):
    
    motion_gen = get_franka_motion_gen(world_config=world_config, gripper_width=gripper_width)
    start_state.joint_names = motion_gen.kinematics.joint_names.copy()
    goal_state.joint_names = motion_gen.kinematics.joint_names.copy()
    
    attached_state.joint_names = motion_gen.kinematics.joint_names.copy()
    motion_gen.attach_external_objects_to_robot(attached_state,
                                                [attached_object],
                                                surface_sphere_radius=0.015,
                                                link_name="attached_object")

    result = motion_gen.plan_single_js(start_state, goal_state, MotionGenPlanConfig(max_attempts=10))
    if result.success.any():
        qtraj = result.get_interpolated_plan().position
        return qtraj
    else:
        return None

### Choose one pick-place-approach pair

In [ ]:
pick_place_id = 0
pick_pose = pick_poses.clone()[pick_place_id]
place_pose = place_poses.clone()[pick_place_id]
pick_approach_pose = pick_approach_poses.clone()[pick_place_id]
place_approach_pose = place_approach_poses.clone()[pick_place_id]

## 4.1 Retract -> pick_approach

### Compute attach objects and states

In [ ]:
pick_result = plan_ik(pick_pose, pick_world[3], 0.04)
pick_q = pick_result[0].cpu().numpy().reshape(1, -1)

attached_state = JointState.from_position(pick_result[0])
attached_object = pick_world[3].mesh[0]
start_state = JointState.from_position(left_kin_model.retract_config.view(1, -1), joint_names=left_kin_model.joint_names)

attached_object_for_rendering = trimesh.Trimesh(pick_world[1][0].vertices.copy(), pick_world[1][0].faces.copy())
T = poses_to_frames(pick_pose).reshape(4, 4)
invT = np.linalg.inv(T)
attached_object_for_rendering.apply_transform(invT)

# render
publish_clear()
publish_robot_trajs(retract_q, 0.04)
publish_grasp_object(attached_object_for_rendering)

### Plan motion

In [ ]:
trajs_retract_pick_approach_result = plan_motion_attached_object(start_state, pick_approach_pose, attached_object, attached_state, ground_world, 0.02)
trajs_retract_pick = trajs_retract_pick_approach_result[0].clone().cpu().numpy()
trajs_retract_pick = np.vstack([trajs_retract_pick, pick_q])

#render
publish_clear()
publish_robot_trajs(trajs_retract_pick, 0.04)
publish_world(pick_world[3], merged = True)

### Resolve IK Jumping

In [ ]:
pick_approach_state = JointState.from_position(trajs_retract_pick_approach_result[0][-1], joint_names=left_kin_model.joint_names)
pick_result = plan_ik_seed_config(pick_pose, pick_world[3], pick_approach_state, 0.04)
pick_q = pick_result[0].cpu().numpy()
trajs_retract_pick = trajs_retract_pick_approach_result[0].clone().cpu().numpy()
trajs_retract_pick = np.vstack([trajs_retract_pick, pick_q])

# render
publish_clear()
publish_robot_trajs(trajs_retract_pick, 0.04)
publish_world(pick_world[3], merged = True)

## 4.2 retract -> place_approach

### Compute attach objects and states

In [ ]:
place_result = plan_ik(place_pose, place_world, 0.04)
place_q = place_result[0].cpu().numpy().reshape(1, -1)
attached_state = JointState.from_position(place_result[0])
attached_object = place_world.mesh[0]
start_state = JointState.from_position(left_kin_model.retract_config.view(1, -1), joint_names=left_kin_model.joint_names)
attached_object_for_rendering = trimesh.Trimesh(assembly_meshes[0].vertices.copy(), assembly_meshes[0].faces.copy())
T = poses_to_frames(place_pose).reshape(4, 4)
invT = np.linalg.inv(T)
attached_object_for_rendering.apply_transform(invT)

publish_clear()
publish_robot_trajs(retract_q, 0.04)
publish_grasp_object(attached_object_for_rendering)

### Plan motion

In [ ]:
trajs_retract_place_approach_result = plan_motion_attached_object(start_state, place_approach_pose, attached_object, attached_state, ground_world, 0.02)
trajs_retract_place = trajs_retract_place_approach_result[0].cpu().numpy()
trajs_retract_place = np.vstack([trajs_retract_place, place_q])

#render
publish_clear()
publish_robot_trajs(trajs_retract_place, 0.02)
publish_grasp_object(attached_object_for_rendering)
publish_world(ground_world, merged = True)

### Resolve IK Jumping

In [ ]:
place_approach_state = JointState.from_position(trajs_retract_place_approach_result[0][-1], joint_names=left_kin_model.joint_names)
place_result = plan_ik_seed_config(place_pose, place_world, place_approach_state, 0.04)
place_q = place_result[0].cpu().numpy()
trajs_retract_place = trajs_retract_place_approach_result[0].clone().cpu().numpy()
trajs_retract_place = np.vstack([trajs_retract_place, place_q])

# render
publish_clear()
publish_robot_trajs(trajs_retract_place, 0.02)
publish_grasp_object(attached_object_for_rendering)
publish_world(ground_world, merged = True)

## 4.2 pick_approach -> place_approach

In [ ]:
pick_approach_q = torch.tensor(trajs_retract_pick[-2, :].reshape(1, -1), **tensor_args.as_torch_dict())
place_approach_q = torch.tensor(trajs_retract_place[-2, :].reshape(1, -1), **tensor_args.as_torch_dict())
start_state = JointState.from_position(pick_approach_q, joint_names=left_kin_model.joint_names)
goal_state = JointState.from_position(place_approach_q, joint_names=left_kin_model.joint_names)

In [ ]:
trajs_pick_place_approach_result = plan_motion_js_attached_object(start_state, goal_state, attached_object, attached_state, ground_world, 0.02)
trajs_pick_place = trajs_pick_place_approach_result.cpu().numpy()
trajs_pick_place = np.vstack([pick_q, trajs_pick_place, place_q])

#render
publish_clear()
publish_robot_trajs(trajs_pick_place, 0.02)
publish_grasp_object(attached_object_for_rendering)
publish_world(ground_world, merged = True)

### Complete Animation

### Retract to pick

In [ ]:
publish_clear()
publish_robot_trajs(trajs_retract_pick, 0.04)
publish_world(pick_world[3], merged = True)

### pick to place

In [ ]:
publish_clear()
publish_robot_trajs(trajs_pick_place, 0.02)
publish_grasp_object(attached_object_for_rendering)
publish_world(ground_world, merged = True)

### place to retract

In [ ]:
publish_clear()
publish_robot_trajs(trajs_retract_place[::-1], 0.04)
publish_world(place_world, merged = True)